# Gffcompare common

Generate one transcriptome with newly assembled tumor-specific transcriptomes, merging all datasets. `gff.combined.loci` will allow us to know those that are shared between patients as well as it's old and new identifiers.


In [ ]:
import os
import pandas as pd
import glob
import numpy as np

GENERAL = "data"

projects = [f.name for f in os.scandir("data") if f.is_dir()]
bash_projects = " ".join(projects)

for proj in projects:
    try:
        os.makedirs(os.path.join("data",proj,"analysis/gffcompare"))
    else:
        print("%s already contains a gffcompare directory" %(proj))
###if projects names are not the GEO ids, we suggest to change them. Keys correspond to project names as they are in the folder and values the GEO id
#dict_projects = {'liver_adjacent_totalRNA':'GSE101432', 'hcc_normal_totalRNA':'GSE77314','zou_hcc_RP_totalRNA':'GSE112705', 'GSE193567':'GSE193567'}
#GEO = dict_projects.values()
#GEO_list = list(GEO)

**TUMOR-SPECIFIC**

Modify identifiers of novel genes adding the patient id

In [ ]:
for proj in project:
    patients=pd.read_csv(os.path.join("data",proj,"results/patients.csv", header=None)

    for p in patients.iloc[:,0]:
        file="data"+proj+"analysis/08_tumor_specific/"+p+"/"+p+"_novel_tumor_specific_genes_1FPKM_300kb.gtf"
        with open (file) as inp:
            outfile="data"+proj+"analysis/gffcompare/"+p+"_novel_tumor_specific_genes_1FPKM_300kb_MODIF.gtf"
            with open(outfile, 'w') as out:
                lines=inp.readlines()
                for line in lines:
                    final_line = line.replace('STRG', 'STRG.'+p)
                    out.write(final_line)


In [ ]:
%%bash -s "$bash_projects"

module load gffcompare/20190227

for proj in $1; do
    declare -a listfiles

    while IFS=, read patient normal tumor; do
        listfiles=(${listfiles[@]} data/$proj/analysis/gffcompare/${patient}_novel_tumor_specific_genes_1FPKM_300kb_MODIF.gtf) # Add new element at the end of the array
    done < data/$proj/results/patients.csv

    cd data/$proj/analysis/gffcompare

    printf "%s\n" "${listfiles[@]}" > list_novel_tumorsp_files_MODIFids.txt

    gffcompare -i list_novel_tumorsp_files_MODIFids.txt -o gffcmp.MODIFids
done

**TOTAL ASSEMBLED NOVEL TRANSCRIPTS**

In [ ]:
%%bash -s "$bash_projects"

module load gffcompare/20190227

for proj in $1; do
    declare -a listfiles

    while IFS=, read patient normal tumor; do
        listfiles=(${listfiles[@]} data/$proj/analysis/06_stringtie/intersect_novel/${normal}_novel.gtf) # Add new element at the end of the array
    done < data/$proj/results/patients.csv

    cd data/$proj/analysis/gffcompare

    printf "%s\n" "${listfiles[@]}" > list_novel_assembled_files.txt

    gffcompare -i list_novel_assembled_files.txt -o gffcmp.assembled
done